In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

from ast import literal_eval

import random

%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('./retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [174]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    recs = random.choices(items_weights['item_id'], weights=items_weights['weight'], k=n)
    
    return recs

In [175]:
%%time

# расчитаем суммы продаж и переименуем колонку

popularity = data_train.groupby('item_id')['sales_value'] \
                        .sum().reset_index().rename(columns={"sales_value": "weight"})

# заменим нулевые веса ненулевыми значениями
popularity['weight'] = popularity['weight'].replace(0., 0.00000001)

# посчитаем логарифм для каждого веса
popularity['weight'] = popularity['weight'].apply(lambda x: np.log(x))

# нормализуем значения весов в диапазоне от 0 до 1
float_array = popularity['weight'].values.astype(float).reshape(-1, 1)

min_max_scaler = preprocessing.MinMaxScaler()
scaled_array = min_max_scaler.fit_transform(float_array)

popularity['weight'] = scaled_array

# созданим новую колонку с базовыми рандомными взвешенными по выручке рекоммендациями
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popularity, n=5))
result.head(2)

CPU times: user 1.59 s, sys: 35.5 ms, total: 1.62 s
Wall time: 1.63 s


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[826658, 13115937, 973916, 867140, 904092]","[999999, 6534178, 6533889, 1029743, 6534166]","[969977, 859191, 965331, 1026258, 969494]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[952317, 5574045, 1075918, 936924, 961792]","[999999, 6534178, 6533889, 1029743, 6534166]","[907631, 1101173, 1008673, 892221, 8291322]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [157]:
def precision_at_k(df, field_name, k=5):
    
    sum_ = 0
    for index, row in df.iterrows():

        bought_list = np.array(row['actual'])
        recommended_list = np.array(row[field_name])


        bought_list = bought_list  # Тут нет [:k] !!
        recommended_list = recommended_list[:k]

        flags = np.isin(bought_list, recommended_list)

        precision = flags.sum() / len(recommended_list)

        sum_ += precision
    return sum_ / df.shape[0]

In [170]:
precision_at_k(result, 'random_recommendation')

0.006268364348677761

In [173]:
precision_at_k(result, 'popular_recommendation')

0.08560235063663048

In [176]:
precision_at_k(result, 'weighted_random_recommendation')

0.006366307541625851

In [183]:
precision_at_k(result, 'itemitem_k5')

0.033594515181195064

In [184]:
precision_at_k(result, 'itemitem_k10')

0.030460333006856143

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [166]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [167]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

/home/alex/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [168]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [169]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

CPU times: user 473 ms, sys: 3.75 ms, total: 477 ms
Wall time: 475 ms


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[826658, 13115937, 973916, 867140, 904092]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[952317, 5574045, 1075918, 936924, 961792]"


In [171]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [172]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 129 ms, sys: 16.1 ms, total: 146 ms
Wall time: 142 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[826658, 13115937, 973916, 867140, 904092]","[999999, 6534178, 6533889, 1029743, 6534166]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[952317, 5574045, 1075918, 936924, 961792]","[999999, 6534178, 6533889, 1029743, 6534166]"


In [177]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [178]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [179]:
%%time

model = ItemItemRecommender(K=10, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 8.06 s, sys: 36.1 ms, total: 8.1 s
Wall time: 2.43 s


In [180]:
%%time

result['itemitem_k10'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 180 ms, sys: 3.94 ms, total: 184 ms
Wall time: 181 ms


In [181]:
%%time

model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=True, 
                        filter_items=None, 
                        recalculate_user=True)


CPU times: user 7.81 s, sys: 47.5 ms, total: 7.86 s
Wall time: 2.35 s


In [182]:
%%time

result['itemitem_k5'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=True, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 153 ms, sys: 3.86 ms, total: 157 ms
Wall time: 156 ms
